In [1]:
# tf 1.0.1 cant do time-distributed conv2d, need to get cuda6/cudnn5 and tf 1.4
import h5py
import numpy as np

datafile = h5py.File('SCNeuronModelCompetition.mat')
movie = datafile.get('trainingmovie_norm') # normalized movie
frhist = datafile.get('FRhist_tr') # firing rate histograms
movie = np.array(movie).reshape(288, 150, 128, 96, 1)

import keras
from keras.layers import Dense, Dropout, TimeDistributed, Activation
from keras.layers import Conv2D, MaxPool2D, Reshape

model = keras.models.Sequential()

model.add(TimeDistributed(Conv2D(10, (3, 3), activation='relu'),
                          input_shape=movie.shape[1:]))
model.add(TimeDistributed(MaxPool2D((2, 2))))
model.add(TimeDistributed(Reshape((29610,))))
model.add(TimeDistributed(Dense(10, activation='relu')))

model.add(Dense(frhist.shape[2], activation='softplus'))

adamopt = keras.optimizers.Adam(lr=0.001, decay=1e-7)
model.compile(optimizer=adamopt, loss='poisson')
model.summary()

history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True)


ValueError: cannot reshape array of size 1 into shape (288,150,128,96,1)